in order to run deep learning experiments on GPU, you will have to change the runtime to GPU this way:

top menu: Runtime / Change runtime type / Hardware accelerator: GPU



In [2]:
# install requirements:
#  SPTK for speech synthesis,
#  SoX for sound conversion,
#  PySPTK for spectrogram
!apt install sptk
!apt install sox
!pip3 install pysptk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sptk is already the newest version (3.9-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 10 not upgraded.
Need to get

In [3]:
# mount my own google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# check contents of my google drive
import os
os.chdir("/content/gdrive/My Drive/")
!ls -al

total 396608
-rw------- 1 root root   1366740 Jul 13  2015 '2008.12.21 letöltés 001.jpg'
drwx------ 3 root root      4096 Dec  1 19:30  2017-2018-2019
-rw------- 1 root root  41419490 Sep  9  2018  358986411-keynote-upper-intermediate-student-s-book-pdf.pdf
drwx------ 3 root root      4096 Feb  6  2018  4.félév
-rw------- 1 root root         1 Jan 25 15:40 '6. félév órarend.gsheet'
drwx------ 2 root root      4096 Dec 14  2017 'Ádám meglepi'
drwx------ 2 root root      4096 Mar  3 10:50  BEST
drwx------ 2 root root      4096 May  2  2018 'Bogi '
drwx------ 2 root root      4096 Feb 23  2018  boldi_angyalai
-rw------- 1 root root         1 Dec  7 16:09 'BSc szakmai gyakorlat 2018 @ Morgan Stanley.gsheet'
drwx------ 2 root root      4096 Sep 13  2018  Classroom
drwx------ 2 root root      4096 Nov 23 12:29 'Colab Notebooks'
drwx------ 2 root root      4096 Jul 13  2015 'Családi képek'
drwx------ 2 root root      4096 Mar  1 17:14 'Dani felvételi'
drwx------ 3 root root      4

In [5]:
# create new folder for output
dir_output = "/content/gdrive/My Drive/deep_learning_VID2SPEECH/"
if not os.path.exists(dir_output):
  os.mkdir(dir_output)
os.chdir(dir_output)
!pwd

/content/gdrive/My Drive/deep_learning_VID2SPEECH


---

functions that will be used later


In [0]:


from subprocess import run
import numpy as np
import scipy.io.wavfile as io_wav

def encode(basefilename, Fs_new = 11050, frlen = 512, frshft = 200, order = 12, alpha = 0.42, stage = 3, minF0 = 50, maxF0 = 400):

    # from HTS Makefile
    # $(X2X) +sf $${raw} | $(PITCH) -H $(UPPERF0) -L $(LOWERF0) -p $(FRAMESHIFT) -s $${SAMPKHZ} -o 2 > lf0/$${base}.lf0
    #
    # $(X2X) +sf $${raw} | \
    # $(FRAME) -l $(FRAMELEN) -p $(FRAMESHIFT) | \
    # $(WINDOW) -l $(FRAMELEN) -L $(FFTLEN) -w $(WINDOWTYPE) -n $(NORMALIZE) | \
    # $(MGCEP) -a $(FREQWARP) -c $(GAMMA) -m $(MGCORDER) -l $(FFTLEN) -e 1.0E-08 -o 4 | \
    # $(LPC2LSP) -m $(MGCORDER) -s $${SAMPKHZ} $${GAINOPT} -n $(FFTLEN) -p 8 -d 1.0E-08 > mgc/$${base}.mgc; \

    # calculate MGC-LSP
    command = 'sox "' + basefilename + '.wav"' + ' -t raw -r ' + str(Fs_new) + ' - ' + ' | sptk x2x +sf | ' + \
              'sptk frame -l ' + str(frlen) + ' -p ' + str(frshft) + ' | ' + \
              'sptk window -l ' + str(frlen) + ' -L ' + str(frlen) + ' -w 0 -n 1 | ' + \
              'sptk mgcep -a ' + str(alpha) + ' -c ' + str(stage) + ' -m ' + str(order) + ' -l ' + str(frlen) + ' -e 1.0E-08 -o 4 | ' + \
              'sptk lpc2lsp -m ' + str(order) + ' -s ' + str(Fs_new / 1000) + ' -n ' + str(frlen) + ' -p 8 -d 1.0E-08 > "' + basefilename + '.mgclsp"'
    # print(command)
    run(command, shell=True)

    # estimate pitch using SWIPE
    command = 'sox "' + basefilename + '.wav"' + ' -t raw -r ' + str(Fs_new) + ' - ' + ' | sptk x2x +sf | ' + \
              'sptk pitch -a 1 -H ' + str(maxF0) + ' -L ' + str(minF0) + ' -p ' + str(frshft) + ' -s ' + str(Fs_new / 1000) + ' -o 2 > "' + basefilename + '.lf0"'
    # print(command)
    run(command, shell=True)
    
    # read files for output
    mgc_lsp_coeff = np.fromfile(basefilename + '.mgclsp', dtype=np.float32).reshape(-1, order + 1)
    lf0 = np.fromfile(basefilename + '.lf0', dtype=np.float32)

    return (mgc_lsp_coeff, lf0) 



In [0]:
import scipy.io.wavfile as io_wav

def decode(mgc_lsp_coeff, lf0, basefilename_out, Fs = 11050, frlen = 512, frshft = 200, order = 12, alpha = 0.42, stage = 3):
    
    # from HTS Training.pl / gen_wave
    #
    # MGC-LSPs -> MGC coefficients
    # $line = "$LSPCHECK -m " . ( $ordr{'mgc'} - 1 ) . " -s " . ( $sr / 1000 ) . " $lgopt -c -r 0.1 -g -G 1.0E-10 $mgc | ";
    # $line .= "$LSP2LPC -m " . ( $ordr{'mgc'} - 1 ) . " -s " . ( $sr / 1000 ) . " $lgopt | ";
    # $line .= "$MGC2MGC -m " . ( $ordr{'mgc'} - 1 ) . " -a $fw -c $gm -n -u -M " . ( $ordr{'mgc'} - 1 ) . " -A $fw -C $gm " . " > $gendir/$base.c_mgc";
    # shell($line);
    #
    # $line = "$SOPR -magic -1.0E+10 -EXP -INV -m $sr -MAGIC 0.0 $lf0 > $gendir/${base}.pit";
    #
    # $line = "$EXCITE -n -p $fs $gendir/$base.pit | ";
    # $line .= "$DFS -b $lfil | $VOPR -a $gendir/$base.unv | ";
    # $line .= "$MGLSADF -P 5 -m " . ( $ordr{'mgc'} - 1 ) . " -p $fs -a $fw -c $gm $mgc | ";
    # $line .= "$X2X +fs -o > $gendir/$base.raw";
    # shell($line);
    
    # write files for SPTK
    mgc_lsp_coeff.astype('float32').tofile(basefilename_out + '.mgclsp')
    lf0.astype('float32').tofile(basefilename_out + '.lf0')
    
    # MGC-LSPs -> MGC coefficients
    command = 'sptk lspcheck -m ' + str(order) + ' -s ' + str(Fs / 1000) + ' -c -r 0.1 -g -G 1.0E-10 "' + basefilename_out + '.mgclsp"' + ' | ' + \
              'sptk lsp2lpc -m '  + str(order) + ' -s ' + str(Fs / 1000) + ' | ' + \
              'sptk mgc2mgc -m '  + str(order) + ' -a ' + str(alpha) + ' -c ' + str(stage) + ' -n -u ' + \
                      '-M '  + str(order) + ' -A ' + str(alpha) + ' -C ' + str(stage) + ' > "' + basefilename_out + '.mgc"'
   # print(command)
    test = run(command, shell=True)
    print(test)
    # MGLSADF synthesis based on pitch and MGC coefficients
    command = 'sptk sopr -magic -1.0E+10 -EXP -INV -m ' + str(Fs) + ' -MAGIC 0.0 "' + basefilename_out + '.lf0"' + ' | ' + \
              'sptk excite -n -p ' + str(frshft) + ' | ' + \
              'sptk mglsadf -P 5 -m ' + str(order) + ' -p ' + str(frshft) + ' -a ' + str(alpha) + ' -c ' + str(stage) + ' "' + basefilename_out + '.mgc"' + ' | ' + \
              'sptk x2x +fs -o | sox -c 1 -b 16 -e signed-integer -t raw -r ' + str(Fs) + ' -t wav -r ' + str(Fs) + ' "' + basefilename_out + '.wav"'
    # print(command)
    test2 = run(command, shell=True)
    print(test2)
    # read file for output
    (Fs_out, x_synthesized) = scipy.io.wavfile.read(basefilename_out + '.wav')
    
    return x_synthesized


In [8]:
# from LIP_SSI_DNN_GRID.py

# required modules
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as io_wav
import os
import os.path
import gc
import re
import csv
import datetime
import scipy
import pickle
import cv2

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, CSVLogger

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, IncrementalPCA

Using TensorFlow backend.


In [0]:
# from LipReading
################## VIDEO INPUT WITH DIFF ##################
def load_video_3D(path):
    
    cap = cv2.VideoCapture(path)
    frameCount = THRESH_FRAME_COUNT
    frameHeight=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
    frameWidth=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))

    buf =np.empty((frameHeight, frameWidth, frameCount), np.dtype('float32'))
    fc = 0
    ret = True
    
    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame=frame.astype('float32')
        #frame = frame-np.mean(frame)
        #frame = frame/np.std(frame)
        frame = frame-np.amin(frame)
        frame = frame/np.amax(frame)
        buf[:,:,fc]=frame
        fc += 1
    cap.release()

    return buf

In [0]:
def get_mgc_lsp_coeff(basefilename):
    if os.path.isfile(basefilename + '.mgclsp'):
        mgc_lsp_coeff = np.fromfile(basefilename + '.mgclsp', dtype=np.float32).reshape(-1, order + 1)
        lf0 = np.fromfile(basefilename + '.lf0', dtype=np.float32)
    else:
        Fs_new = 8000
        (mgc_lsp_coeff, lf0) = encode(basefilename, Fs_new, frameLength, frameShift, order, alpha, stage)
    return (mgc_lsp_coeff, lf0)


In [11]:
# download data and extract
if not os.path.isfile('GRID_s3_cut.tar.gz'):
  !wget http://smartlab.tmit.bme.hu/csapo/GRID/GRID_s3_cut.tar.gz
  !tar -xzvf GRID_s3_cut.tar.gz
!ls -al GRID/

total 8
drwx------ 3 root root 4096 Feb 19 14:57 Audio
drwx------ 3 root root 4096 Feb 19 14:57 Video


In [0]:
# TODO: modify this according to your data path
speaker = 's3'
dir_video = dir_output + 'GRID/Video/' + speaker + '/face/'
dir_audio = dir_output + 'GRID/Audio/' + speaker + '/'


In [0]:
# Parameters of vocoder
samplingFrequency = 8000
frameLength = 1024 # 93 ms at 22050 Hz sampling
frameShift = 320 # 40 ms at 8000 Hz sampling, correspondong to 25 fps (lip video)
order = 12
alpha = 0.42
stage = 3
THRESH_FRAME_COUNT = 75

In [0]:
n_files = 100
n_file = 0
n_max_lip_frames = n_files * THRESH_FRAME_COUNT
n_lines = 128
n_pixels = 128
n_lines_reduced = 128
n_pixels_reduced = 128
n_mgc = order + 1
n_lip = n_lines_reduced * n_pixels_reduced
n_pca = 200
lip = np.empty((n_max_lip_frames, n_lip))
mgc = np.empty((n_max_lip_frames, n_mgc))
lip_size = 0
mgc_size = 0
NFRAMES = 4
COLORS = 1
CHANNELS = COLORS*NFRAMES

In [0]:
train_rate = 0.8
valid_rate = 0.2

In [16]:
if os.path.isdir(dir_video):
    for file in sorted(os.listdir(dir_video)):
        if ".avi" in file and n_file < n_files: # and "0088" in file:
            try:
                lip_data = load_video_3D(dir_video + file)
                (mgc_lsp_coeff, lf0) = get_mgc_lsp_coeff(dir_audio + file[:-4])
            except ValueError as e:
                print("wrong data, check manually!", e)
            
            else:
                n_file += 1
                
                mgc_lip_len = np.min((lip_data.shape[2], len(mgc_lsp_coeff)))
                
                lip_data = lip_data[:, :, 0:mgc_lip_len]
                mgc_lsp_coeff = mgc_lsp_coeff[0:mgc_lip_len]
                
                print(file[:-4], lip_data.shape, mgc_lsp_coeff.shape)
                
                # resize
                lip_len = lip_data.shape[2]
                if lip_size + lip_len > n_max_lip_frames:
                    raise
                for i in range(lip_len):
                    # lip[lip_size + i] = scipy.misc.imresize(lip_data[i], (n_lines_reduced, n_pixels_reduced), interp='bicubic') # 64x64
                    lip[lip_size + i] = lip_data[:, :, i].ravel() # original, 128x128
                    mgc[mgc_size + i] = mgc_lsp_coeff[i]

                lip_size += lip_len
                mgc_size += lip_len
                
                print('n_frames_all: ', lip_size, 'mgc_size: ', mgc_size)
                # calculate mgc features

lip = lip[0 : lip_size]
mgc = mgc[0 : mgc_size]

000001 (128, 128, 0) (0, 13)
n_frames_all:  0 mgc_size:  0
000002 (128, 128, 75) (75, 13)
n_frames_all:  75 mgc_size:  75
000003 (128, 128, 75) (75, 13)
n_frames_all:  150 mgc_size:  150
000004 (128, 128, 75) (75, 13)
n_frames_all:  225 mgc_size:  225
000005 (128, 128, 75) (75, 13)
n_frames_all:  300 mgc_size:  300
000006 (128, 128, 75) (75, 13)
n_frames_all:  375 mgc_size:  375
000007 (128, 128, 75) (75, 13)
n_frames_all:  450 mgc_size:  450
000008 (128, 128, 75) (75, 13)
n_frames_all:  525 mgc_size:  525
000009 (128, 128, 75) (75, 13)
n_frames_all:  600 mgc_size:  600
000010 (128, 128, 75) (75, 13)
n_frames_all:  675 mgc_size:  675
000011 (128, 128, 75) (75, 13)
n_frames_all:  750 mgc_size:  750
000012 (128, 128, 75) (75, 13)
n_frames_all:  825 mgc_size:  825
000013 (128, 128, 75) (75, 13)
n_frames_all:  900 mgc_size:  900
000014 (128, 128, 75) (75, 13)
n_frames_all:  975 mgc_size:  975
000015 (128, 128, 75) (75, 13)
n_frames_all:  1050 mgc_size:  1050
000016 (128, 128, 75) (75, 13)


In [17]:
print(lip.shape[0])

print(lip_size)

7425
7425


In [18]:
print(lip.shape, mgc.shape)

samples = np.zeros((lip_size), dtype=[('input', float, (CHANNELS ,n_lines, n_pixels)), ('output', float, (n_mgc))])

for i in range(0, lip_size-CHANNELS+1):
    temp_in = np.zeros((CHANNELS, n_lines, n_pixels))
    for j in range(0, CHANNELS):
        temp_in[j] = lip[i+j].reshape(n_lines, n_pixels)
    samples[i] = temp_in, mgc[i]

(7425, 16384) (7425, 13)


In [19]:
samples[0]['input'].shape

(4, 128, 128)

In [20]:
print(samples['input'].shape)
print(samples[0]['input'].shape)
print(samples[0]['input'])
print(samples[1]['input'])

(7425, 4, 128, 128)
(4, 128, 128)
[[[0.79581153 0.8062827  0.81151831 ... 0.539267   0.54973823 0.539267  ]
  [0.79057592 0.79581153 0.80104715 ... 0.53403139 0.54450262 0.54450262]
  [0.78534031 0.77486908 0.79057592 ... 0.52879584 0.539267   0.55497384]
  ...
  [0.55497384 0.54450262 0.51832461 ... 0.14136125 0.14136125 0.14659686]
  [0.49214661 0.4764398  0.44502619 ... 0.13089006 0.13612565 0.13612565]
  [0.41884816 0.39790577 0.37172773 ... 0.12565444 0.12565444 0.12565444]]

 [[0.78534031 0.79581153 0.8062827  ... 0.539267   0.54973823 0.539267  ]
  [0.78534031 0.79057592 0.80104715 ... 0.53403139 0.54450262 0.54450262]
  [0.79057592 0.78534031 0.79057592 ... 0.52879584 0.539267   0.55497384]
  ...
  [0.54973823 0.54973823 0.52879584 ... 0.14136125 0.14136125 0.14659686]
  [0.49738219 0.47120419 0.43979058 ... 0.13089006 0.13612565 0.13612565]
  [0.42408377 0.39267015 0.36649215 ... 0.12565444 0.12565444 0.12565444]]

 [[0.79057592 0.80104715 0.81151831 ... 0.539267   0.54973823 

In [21]:
lip_size

7425

In [0]:
train_samples = np.zeros(int(train_rate*lip_size), dtype=[('input', float, (CHANNELS, n_lines, n_pixels)), ('output', float, (n_mgc))])
valid_samples = np.zeros(int(valid_rate*lip_size), dtype=[('input', float, (CHANNELS, n_lines, n_pixels)), ('output', float, (n_mgc))])

# use 90% of data for training, 10% for validation
train_samples = samples[0 : int(train_rate*lip_size)]
valid_samples = samples[int(valid_rate*lip_size) : ]
#mgc_training = mgc[0 : int(0.9*mgc_size)]
#mgc_validation = mgc[int(0.9*mgc_size) : ]



In [0]:
xtrain_mean = np.mean(train_samples['input'])
train_samples['input'] = train_samples['input'] - xtrain_mean
valid_samples['input'] = valid_samples['input'] - xtrain_mean

ytrain_mean = np.mean(train_samples['output'])
y_stds = np.std(train_samples['output'])

ytrain_norm =((train_samples['output']-ytrain_mean)/y_stds)
yvalid_norm =((valid_samples['output']-ytrain_mean)/y_stds)



In [0]:
DROPOUT = 0.25
DROPOUT2 = 0.5

In [26]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Reshape
from keras.layers import TimeDistributed
from keras.layers import LeakyReLU
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Conv2D(input_shape=(CHANNELS ,n_lines, n_pixels), filters=32, kernel_size=(3,3), strides=(1, 1), padding='same', use_bias=True, data_format= "channels_first"))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2),  padding='same', data_format= "channels_first"))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same', use_bias=True, data_format= "channels_first"))
model.add(BatchNormalization())
model.add(LeakyReLU())	
model.add(MaxPooling2D(pool_size=(2, 2),  padding='same', data_format= "channels_first"))
model.add(Dropout(DROPOUT))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same', use_bias=True, data_format= "channels_first"))
model.add(BatchNormalization())
model.add(LeakyReLU())	
model.add(MaxPooling2D(pool_size=(2, 2),  padding='same', data_format= "channels_first"))
model.add(Dropout(DROPOUT2))
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Dense(13))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [27]:
from keras.optimizers import Adam
adam=Adam(lr=0.01)
# compile model
model.compile(loss='mean_squared_error', optimizer=adam)

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 128, 128)      1184      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 128, 128)      512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 128, 128)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 64, 64)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 64, 64)        0         
__________

In [28]:
# early stopping to avoid over-training
# csv logger
current_date = '{date:%Y-%m-%d_%H-%M-%S}'.format( date=datetime.datetime.now() )
print(current_date)
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0), \
             CSVLogger(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '.csv', append=True, separator=';')]

2019-03-27_20-58-46


In [29]:
# Run training
history = model.fit(train_samples['input'], ytrain_norm,
                        epochs = 60, batch_size = 32, shuffle = True, verbose = 1,
                        validation_data=(valid_samples['input'], yvalid_norm),
                        callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 5940 samples, validate on 5940 samples
Epoch 1/60
5940/5940 [==============================] - 24s 4ms/step - loss: 1.5246 - val_loss: 0.5713
Epoch 2/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.6517 - val_loss: 0.5766
Epoch 3/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.5643 - val_loss: 0.5084
Epoch 4/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.5233 - val_loss: 0.5616
Epoch 5/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.5031 - val_loss: 0.4637
Epoch 6/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.5202 - val_loss: 0.5018
Epoch 7/60
5940/5940 [==============================] - 20s 3ms/step - loss: 0.4874 - val_loss: 0.4416
Epoch 8/60
5940/5940 [==============================] - 20s 3ms/step - loss: 0.4825 - val_loss: 0.4623
Epoch 9/60
5940/5940 [==============================] - 19s 3ms/step - loss: 0.4

In [0]:
model_json = model.to_json()
with open(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '_model.json', "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '_weights.h5')

# serialize scalers to pickle
#pickle.dump(lip_pca, open(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '_lip_pca.sav', 'wb'))
#pickle.dump(lip_scaler, open(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '_lip_scaler.sav', 'wb'))
#pickle.dump(mgc_scalers, open(dir_output + 'VID2SPEECH_GRID_baseline_' + current_date + '_mgc_scalers.sav', 'wb'))

In [31]:
# evaluate on test data, from the end of file list
n_files = 2
n_file = 0
if os.path.isdir(dir_video):
    for file in sorted(os.listdir(dir_video)):
        if ".avi" in file and n_file < n_files and "000990" in file:
            print('testing on: ', file)
            lip_test = np.empty((75, n_lip))
            lip_data = load_video_3D(dir_video + file)
            lip_len = lip_data.shape[2]
            (mgc_lsp_coeff, lf0) = get_mgc_lsp_coeff(dir_audio + file[:-4])
            for i in range(lip_len):
                # lip[lip_size + i] = scipy.misc.imresize(lip_data[i], (n_lines_reduced, n_pixels_reduced), interp='bicubic') # 48x64
                lip_test[i] = lip_data[:, :, i].ravel() # original, 128x128
            

            #lip_test = lip_scaler.transform(lip_test)
            print(lip_test.shape)
            lip_test_shape = lip_test.shape
            input_test_data = np.zeros((lip_test_shape[0], CHANNELS, 128, 128))
            
            
            for i in range(0, len(lip_test)-CHANNELS+1):
                temp_in = np.zeros((CHANNELS, 128, 128))
                for j in range(0, CHANNELS):
                    temp_in[j] = cv2.resize(lip_test[i+j],(n_lines, n_pixels))
                input_test_data[i] = temp_in
            
            
            
            
            
            #lip_test = np.reshape(lip_test, (-1,1,200))
            # predict using the trained model
            mgc_predicted = model.predict(input_test_data)
            
           # mgc_predicted_ = np.zeros((lip_test_shape[0], 13))
            #for i in range(n_mgc):
             #   mgc_predicted[:, i] = mgc_scalers[i].inverse_transform(mgc_predicted[:, 0, i].reshape(-1, 1)).ravel()
                    
            n_file += 1
            
            lf0_whisper = np.ones((len(lf0))) * -1e10      
            print(mgc_predicted.shape)
            print(mgc_predicted[0].shape)
            print(mgc_predicted[29])
            #decode(mgc_lsp_coeff, lf0, dir_output + file[:-4] + '_original_vocoded', Fs = 8000, frlen = frameLength, frshft = frameShift, order = order, alpha = alpha, stage = stage)
            decode(mgc_predicted, lf0, dir_output + file[:-4] +'_' +current_date + '_LSTM_predicted_PCA', Fs = 8000, frlen = frameLength, frshft = frameShift, order = order, alpha = alpha, stage = stage)
            #decode(mgc_lsp_coeff, lf0_whisper, dir_output + file[:-4] + '_original_vocoded_whisper', Fs = 8000, frlen = frameLength, frshft = frameShift, order = order, alpha = alpha, stage = stage)
            #decode(mgc_predicted, lf0_whisper, dir_output + file[:-4] +'_' + current_date + '_CNN3D_predicted_whisper', Fs = 8000, frlen = frameLength, frshft = frameShift, order = order, alpha = alpha, stage = stage)

testing on:  000990.avi
(75, 16384)
(75, 13)
(13,)
[-2.7348208  -0.20517637 -0.14295322 -0.01999933 -0.1741185  -0.16514981
 -0.13031638 -0.15229058 -0.17233938 -0.1645992  -0.13064483 -0.16217467
 -0.12960622]
CompletedProcess(args='sptk lspcheck -m 12 -s 8.0 -c -r 0.1 -g -G 1.0E-10 "/content/gdrive/My Drive/deep_learning_VID2SPEECH/000990_2019-03-27_20-58-46_LSTM_predicted_PCA.mgclsp" | sptk lsp2lpc -m 12 -s 8.0 | sptk mgc2mgc -m 12 -a 0.42 -c 3 -n -u -M 12 -A 0.42 -C 3 > "/content/gdrive/My Drive/deep_learning_VID2SPEECH/000990_2019-03-27_20-58-46_LSTM_predicted_PCA.mgc"', returncode=0)
CompletedProcess(args='sptk sopr -magic -1.0E+10 -EXP -INV -m 8000 -MAGIC 0.0 "/content/gdrive/My Drive/deep_learning_VID2SPEECH/000990_2019-03-27_20-58-46_LSTM_predicted_PCA.lf0" | sptk excite -n -p 320 | sptk mglsadf -P 5 -m 12 -p 320 -a 0.42 -c 3 "/content/gdrive/My Drive/deep_learning_VID2SPEECH/000990_2019-03-27_20-58-46_LSTM_predicted_PCA.mgc" | sptk x2x +fs -o | sox -c 1 -b 16 -e signed-intege

FileNotFoundError: ignored

In [0]:
# listen to original sentence
import IPython

(Fs, x) = io_wav.read(dir_output + '000099_2019-03-16_21-28-32_CNN3D_predicted.wav')
IPython.display.Audio(x, rate=Fs)

In [0]:
# listen to synthesized sentence (using LIP video as input)
(Fs, x) = io_wav.read(dir_output + '000990_DNN_predicted_PCA.wav')
IPython.display.Audio(x, rate=Fs)

In [0]:
# listen to original sentence (whispered)
import IPython

(Fs, x) = io_wav.read(dir_output + '000990_original_vocoded_whisper.wav')
IPython.display.Audio(x, rate=Fs)

In [0]:
# listen to synthesized sentence (whispered)
(Fs, x) = io_wav.read(dir_output + '000990_2019-03-14_06-47-15_CNN_predicted_whisper.wav')
IPython.display.Audio(x, rate=Fs)

In [0]:
# save file to local drive
from google.colab import files
files.download(dir_output + '000990_DNN_predicted_PCA_whisper.wav')

In [0]:
# show original and predicted spectrograms


In [0]:
import pysptk
import librosa
from librosa import display

In [0]:
mgc_original = np.fromfile(dir_output + '000099_2019-03-16_21-28-32_CNN3D_predicted.mgc', dtype=np.float32).reshape(-1, order + 1)
logH_original = pysptk.mgc2sp(np.float64(mgc_original), 0.0, 0.0, frameLength).real
display.specshow(logH_original.T, sr=Fs, hop_length=frameShift, x_axis="time", y_axis="linear")

In [0]:
mgc_predicted = np.fromfile(dir_output + '000990_DNN_predicted_PCA_whisper.mgc', dtype=np.float32).reshape(-1, order + 1)
logH_predicted = pysptk.mgc2sp(np.float64(mgc_predicted), 0.0, 0.0, frameLength).real
display.specshow(logH_predicted.T, sr=Fs, hop_length=frameShift, x_axis="time", y_axis="linear")